In [1]:
from datetime import datetime
print(f'Päivitetty {datetime.now().date()} / Aki Taanila')

Päivitetty 2025-05-17 / Aki Taanila


Ajoneuvojen avoin data sisältää kaikkien liikennekäytössä olevien ajoneuvojen rekisteröinti-, hyväksyntä- ja teknisiä tietoja Traficomin ylläpitämästä liikenneasioiden rekisteristä. Datan laajuus on useita miljoonia rivejä, joten sitä ei voi ladata suoraan esimerkiksi Exceliin. Dataa päivitetään säännöllisesti. Data löytyy osoitteesta:

https://tieto.traficom.fi/fi/tietotraficom/avoin-data?toggle=Ajoneuvojen%20avoin%20data

Samasta osoitteesta löytyy myös aineistokuvaus, muuttujaluettelo Excel-muodossa ja käytetty merkistökoodaus (ISO8859-1). Data on ladattavissa zip-pakettina.

Poimin datasta tiedot vuoden 2020 alusta eteenpäin rekisteröidyistä henkilöautoista. Otan mukaan vain osan muuttujista. Tallennan poimitun datan myöhempää käyttöä varten. Tallennettu data on sen kokoinen että sen voi avata tarvittaessa myös Exceliin.

**Koodin suoritus kestää kauan, koska ison datan lataaminen netistä on hidasta**. Sen jälkeen kun data (Ajoneuvoreksiteri.zip) on ladattu koneelle, niin sen lataamiseen liittyvää koodisolua ei tarvitse enää uudelleen suorittaa.

In [2]:
# Ajoneuvorekisteri.zip -tiedoston lataaminen samaan hakemistoon tämän muistion kanssa
# Tämän solun suoritus kestää kauan, koska ladattava data on iso!

import requests
url = "https://opendata.traficom.fi/Content/Ajoneuvorekisteri.zip"
r = requests.get(url)
with open("Ajoneuvorekisteri.zip",'wb') as file:
    file.write(r.content)

In [3]:
import pandas as pd

In [4]:
# Mukaan otettavat muuttujat
cols = ['ajoneuvoluokka', 'ensirekisterointipvm', 'vari', 'korityyppi', 'omamassa', 
        'ajonKokPituus', 'ajonLeveys', 'ajonKorkeus', 'kayttovoima', 'suurinNettoteho', 
        'sahkohybridi', 'sahkohybridinluokka', 'merkkiSelvakielinen','mallimerkinta', 
        'vaihteisto', 'kaupallinenNimi', 'kunta', 'matkamittarilukema']

# Datan avaus (pandas osaa purkaa zip-paketin)
df = pd.read_csv('Ajoneuvorekisteri.zip', sep=';', encoding='ISO8859-1', usecols=cols, low_memory=False)

In [5]:
# Tietoa muuttujista
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5128155 entries, 0 to 5128154
Data columns (total 18 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ajoneuvoluokka        object 
 1   ensirekisterointipvm  object 
 2   vari                  object 
 3   korityyppi            object 
 4   omamassa              float64
 5   ajonKokPituus         float64
 6   ajonLeveys            float64
 7   ajonKorkeus           float64
 8   kayttovoima           object 
 9   suurinNettoteho       float64
 10  sahkohybridi          object 
 11  sahkohybridinluokka   float64
 12  merkkiSelvakielinen   object 
 13  mallimerkinta         object 
 14  vaihteisto            object 
 15  kaupallinenNimi       object 
 16  kunta                 float64
 17  matkamittarilukema    float64
dtypes: float64(8), object(10)
memory usage: 704.2+ MB


In [6]:
# Poimitaan datasta vain henkilöautot (ajoneuvoluokat M1 ja M1G)
df = df[df['ajoneuvoluokka'].isin(['M1', 'M1G'])]

In [7]:
# Varmistetaan, että päivämäärät tunnistuvat päivämääriksi
df['ensirekisterointipvm'] = pd.to_datetime(df['ensirekisterointipvm'], format='%d.%m.%Y')

In [8]:
# Rajoitutaan vuoden 2020 alusta lähtien rekisteröityihin
df = df[df['ensirekisterointipvm']>'2020-01-01'].sort_values('ensirekisterointipvm')

In [9]:
# Tallennus csv-muotoon
df.to_csv('autot.csv', index=False)

In [10]:
# Näytetään tulosteissa kaikki sarakkeet
pd.set_option.display_columns = None
df

,ajoneuvoluokka,ensirekisterointipvm,vari,korityyppi,omamassa,ajonKokPituus,ajonLeveys,ajonKorkeus,kayttovoima,suurinNettoteho,sahkohybridi,sahkohybridinluokka,merkkiSelvakielinen,mallimerkinta,vaihteisto,kaupallinenNimi,kunta,matkamittarilukema
4147548,M1,2020-01-02,6,AF,1314.0,4346.0,1826.0,1605.0,01,96.0,False,NaN,Honda,HR-V Monikäyttöajoneuvo (AF) 5ov 1498cm3,1,HR-V,740.0,48024.0
3848423,M1,2020-01-02,6,AB,1322.0,4512.0,1844.0,1515.0,01,92.0,False,NaN,Ford,Focus Viistoperä (AB) 4ov 999cm3,1,Focus,790.0,60224.0
4110795,M1,2020-01-02,0,AC,1808.0,4889.0,1832.0,1521.0,01,115.0,True,1.0,"Volkswagen, VW",PASSAT Farmari (AC) 5ov 1395cm3 A,2,PASSAT,290.0,175700.0
4110826,M1,2020-01-02,0,AC,2050.0,4936.0,1890.0,1442.0,01,223.0,True,1.0,Volvo,V90 T8 Twin Engine Farmari (AC) 5ov 1969cm3 A,2,V90 T8 Twin Engine,853.0,78201.0
4161482,M1,2020-01-02,8,AC,1665.0,4600.0,1855.0,1685.0,01,131.0,True,2.0,Toyota,TOYOTA RAV4 Farmari (AC) 4ov 2487cm3,3,TOYOTA RAV4,609.0,35766.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5126161,M1,2025-03-31,0,AA,2525.0,4964.0,1961.0,1508.0,04,NaN,False,NaN,Mercedes-Benz,AMG EQE 53 4MATIC+ Sedan (AA) 4ov,Y,AMG EQE 53 4MATIC+,109.0,9109.0
5126164,M1,2025-03-31,0,SH,2885.0,5986.0,2040.0,2365.0,02,130.0,False,NaN,MAN,TGE Pyörätuolin käyttäjälle tarkoitettu ajoneu...,2,TGE,398.0,12.0
5122614,M1,2025-03-31,6,AB,1978.0,4488.0,1884.0,1654.0,04,NaN,False,NaN,Skoda,ELROQ 60 Viistoperä (AB) 5ov A,2,ELROQ 60,935.0,NaN
5103543,M1,2025-03-31,2,AC,2132.0,4745.0,1890.0,1680.0,01,141.0,True,1.0,Mazda,MAZDA CX-60 Farmari (AC) 4ov 2488cm3 A,2,MAZDA CX-60,169.0,NaN


In [11]:
# Värit selviävät aineistokuvauksesta
varit = ['Musta', 'Ruskea (beige)', 'Punainen', 'Oranssi','Keltainen', 'Vihreä', 'Sininen',
        'Violetti', 'Harmaa', 'Valkoinen', 'Monivär.', 'Hopea', 'Turkoosi']

In [12]:
# Kuinka suosittuja ovat eri värit olleet?
df1 = pd.crosstab(df['vari'], 'f')
df1.columns.name = ''

# Sijoitan värien nimet indeksiin, jossa alunperin on värien koodit
df1.index = varit

# Värit suosituimmuusjärjestyksessä
df1.sort_values('f', ascending = False)

,f
Valkoinen,163957
Harmaa,138748
Musta,129870
Sininen,71708
Punainen,50975
Hopea,31197
Ruskea (beige),19479
Vihreä,10470
Oranssi,6748
Keltainen,5272


In [13]:
# 20 suosituinta merkkiä
df2 = pd.crosstab(df['merkkiSelvakielinen'], 'f').nlargest(20, 'f' )
df2.columns.name = ''
df2

,f
merkkiSelvakielinen,
Toyota,83543
Volvo,69694
"Volkswagen, VW",58584
Mercedes-Benz,57331
Skoda,49427
BMW,44034
Kia,40575
Audi,27177
Tesla,26325


Jotkin automerkit saattavat esiintyä tietokannassa useilla eri nimillä. Esimerkiksi Volkswagen esiintyy ainakin kahdella eri nimellä (Volkswagen ja Volkswagen, VW). Seuraavassa yhdistän molemmat saman nimen alle.

In [14]:
df['merkkiSelvakielinen'] = df['merkkiSelvakielinen'].replace({'Volkswagen, VW':'Volkswagen'})
df3 = pd.crosstab(df['merkkiSelvakielinen'], 'f').nlargest(20, 'f' )
df3.columns.name = ''
df3

,f
merkkiSelvakielinen,
Toyota,83543
Volvo,69694
Volkswagen,62646
Mercedes-Benz,57331
Skoda,49427
BMW,44034
Kia,40575
Audi,27177
Tesla,26325


In [15]:
# Rajoitutaan vai suosituimpiin merkkeihin
df_suositut = df[df['merkkiSelvakielinen'].isin(df3.index)]

# Värit suosituilla merkeillä
df4 = pd.crosstab(df_suositut['merkkiSelvakielinen'], df_suositut['vari'])

# Sijoitan värien nimet sarakkeisiin
df4.columns = varit

# Värjätään kunkin merkin kohdalla suosituin väri
df4.style.highlight_max(color='lightgreen', axis=1)

,Musta,Ruskea (beige),Punainen,Oranssi,Keltainen,Vihreä,Sininen,Violetti,Harmaa,Valkoinen,Monivär.,Hopea,Turkoosi
merkkiSelvakielinen,,,,,,,,,,,,,
Audi,7825,391,1129,38,143,479,3043,21,6733,6375,4,887,16
BMW,15089,614,1127,419,39,851,5254,30,8355,11002,10,1001,24
Citroen,832,204,799,24,4,370,734,1,1448,2461,0,31,3
Ford,3842,123,3961,118,73,244,4469,6,4949,5489,5,144,2
Hyundai,2013,909,2065,110,132,721,2318,1,3769,4463,0,786,233
Kia,5712,619,4824,331,414,1704,5941,6,7767,9819,1,3390,1
Mazda,620,58,1729,2,19,4,390,1,1227,1073,1,231,0
Mercedes-Benz,20128,970,2314,12,564,568,3932,35,14516,11831,13,1986,4
Mitsubishi,2739,667,1739,13,14,7,489,5,2404,3543,1,341,2


## Lisätietoa

Data-analytiikka Pythonilla https://tilastoapu.wordpress.com/python/